In [2]:
from collections import defaultdict
import numpy as np

In [73]:
# CSA helpers

def search_next_departure(stop_departures, time):
    '''
        Search the first departure after moment time in the same stop through binary search in the list of S[stop]
    '''
    start = 0
    end = len(stop_departures) - 1

    next_departure_index = -1
    while start <= end:
        mid = (start + end) // 2;

        # Search in the right side
        if (stop_departures[mid][0] < time):
            start = mid + 1

        # Search in the left side
        else:
            next_departure_index = mid
            end = mid - 1
     
    
    # Search is always successful because of the last np.inf departure
    next_departure_time = stop_departures[next_departure_index][0]
    next_departure_pair = stop_departures[next_departure_index]
        
    return next_departure_index, next_departure_pair, next_departure_time

def shift(vector):
    # TODO: include max for earliest arrival
    new_vector = vector.copy()
    new_vector[1:] = vector[:-1]
    new_vector[0] = np.inf
    return new_vector

def minimize(vector1, vector2):
    return np.minimum(vector1, vector2)
    

In [95]:
def CSA(timetable, source_stop, target_stop, min_departure_time, max_arrival_time, max_connections):
    '''
        Implementation of Pareto Connection Scan profile algorithm with interstop footpaths (https://arxiv.org/pdf/1703.05997.pdf)
    '''
    # timetable
    stops, connections, trips, footpaths = timetable
    S = defaultdict(lambda: [(np.inf, np.ones(max_connections) * np.inf)]) # entries of the type       stop: [(departure_time, [arrival_time_with_1_connection, ..., arrival_time_with_max_connections])]
    T = defaultdict(lambda: np.ones(max_connections) * np.inf)
    
    for connection in connections:
        c_dep_stop, c_arr_stop, c_dep_time, c_arr_time, c_trip = connection
        
        # PHASE 1: FIND tau_c (arrival times given this connection)
        
        # Arrival times by walk from c_arr_stop 
        # TODO (right now it is just a place-holder)
        tau_1 = np.ones(max_connections) * c_arr_time if c_arr_stop == target_stop else np.inf
        
        # Arrival times by continuing on the same trip
        tau_2 = T[c_trip]
        
        # Arrival times by moving to another trip
        # TODO add walk time
        tau_3 = search_next_departure(S[c_arr_stop], c_arr_time)[1][1]
        tau_3 = shift(tau_3)
        
        # Find minimum per number of changes between tau_1, tau_2 and tau_3
        tau_c = minimize(minimize(tau_1, tau_2), tau_3)
        
        # PHASE 2: UPDATE S
        
        # arrivals of earliest departure after c_dep_time from stop c_dep_stop
        next_departure_index, next_departure_pair, _ = search_next_departure(S[c_dep_stop], c_dep_time)
        y = next_departure_pair[1]
        new_min_arrivals = minimize(y, tau_c)
        if not np.array_equal(y, new_min_arrivals):
            # add another pair before the next one
            S[c_dep_stop].insert(next_departure_index, (c_dep_time, new_min_arrivals))
        
        # PHASE 2: UPDATE T
        
        T[c_trip] = tau_c
        
    return S[source_stop]

In [96]:
paper_conn = [
    ("y", "t", 10, 11, "1"),
    ("z", "t", 9, 12, "2"),
    ("x", "t", 8, 13, "3"),
    ("x", "y", 8, 9, "4"),
    ("s", "z", 7, 8, "5"),
    ("s", "x", 6, 7, "6"),
    ("s", "t", 5, 14, "7")
]
paper_timetable = (None, paper_conn, None, None)

CSA(paper_timetable, "s", "t", 4, 15, 3)

[(5, array([14., 12., 11.])),
 (6, array([inf, 12., 11.])),
 (7, array([inf, 12., 12.])),
 (inf, array([inf, inf, inf]))]